
**Updating customer records in a SQL Server database with the latest information.	
SCD Type-1,2,3,4,5 Mapping **


Import and Connection Setup


In [2]:
import pyodbc
import pandas as pf
from datetime import datetime

In [3]:
conn=pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL SERVER};"
    "SERVER=DESKTOP-RJ6N7AA\MSSQLSERVER1;"
    "DATABASE=scd_types;"
    "UID=sa;"
    "PWD=1234567890;"
    "TrustServerCertificate=yes;"
)

In [4]:
cursor = conn.cursor()

Create customers table and insert values

In [5]:
cursor.execute("""
    CREATE TABLE Customers (
        CustomerID INT PRIMARY KEY,
        Name NVARCHAR(100),
        Email NVARCHAR(100),
        Membership NVARCHAR(50),
        Region NVARCHAR(50),
        JoinDate DATE
    )
""")


ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'Customers' in the database. (2714) (SQLExecDirectW)")

In [6]:
cursor.executemany("""
    INSERT INTO Customers (CustomerID, Name, Email, Membership, Region, JoinDate)
    VALUES (?, ?, ?, ?, ?, ?)
""", [
    (101, 'John Doe', 'john@example.com', 'Basic', 'East', '2020-01-15'),
    (102, 'Jane Smith', 'jane@example.com', 'Premium', 'West', '2019-11-20'),
    (103, 'Mike Johnson', 'mike@example.com', 'Basic', 'North', '2021-03-10')
])
conn.commit()
conn.close()

IntegrityError: ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of PRIMARY KEY constraint 'PK__Customer__A4AE64B88F11BA50'. Cannot insert duplicate key in object 'dbo.Customers'. The duplicate key value is (101). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621)")

SCD Type 0 

In [7]:
# Before update, fetch original value
cursor.execute("SELECT JoinDate FROM Customers WHERE CustomerID = ?", (101,))
old_joindate = cursor.fetchone()[0]

if old_joindate != datetime.strptime("2023-01-01", "%Y-%m-%d").date():
    print("JoinDate cannot be changed (Type 0)")
else:
    # Safe update for other fields
    cursor.execute("UPDATE Customers SET Region = ? WHERE CustomerID = ?", ('South', 101))
    conn.commit()

JoinDate cannot be changed (Type 0)


Type 1 – Overwrite Value (no history)

In [9]:
cursor.execute("""
    UPDATE Customers
    SET Name=?,Email=?, Membership=?
    Where CustomerID=?
""",('Deepak K','deepak_new@example.com','Platinum',101))
conn.commit()

Type 2 – Keep Full History

In [10]:
cursor.execute("""
IF OBJECT_ID('Customers_History', 'U') IS NULL
CREATE TABLE Customers_History (
    HistoryID INT IDENTITY(1,1) PRIMARY KEY,
    CustomerID INT,
    Name NVARCHAR(100),
    Email NVARCHAR(100),
    Membership NVARCHAR(50),
    Region NVARCHAR(50),
    JoinDate DATE,
    StartDate DATE,
    EndDate DATE,
    IsCurrent BIT
)
""")
conn.commit()

In [13]:
# Fetch current record
cursor.execute("SELECT * FROM Customers WHERE CustomerID = ?", (101,))
row = cursor.fetchone()
# Mark previous as history
cursor.execute("""
    INSERT INTO Customers_History (
        CustomerID, Name, Email, Membership, Region, JoinDate, StartDate, EndDate, IsCurrent
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
""", (
    row.CustomerID, row.Name, row.Email, row.Membership, row.Region, row.JoinDate,
    datetime.now().date(), datetime.now().date(), 0
))

# Update main table
cursor.execute("""
    UPDATE Customers
    SET Name = ?, Email = ?, Membership = ?
    WHERE CustomerID = ?
""", ('Deepak K Updated', 'deepak_updated@example.com', 'Diamond', 101))
conn.commit()
print("SCD Type 2: Historical data saved.")


SCD Type 2: Historical data saved.


SCD Type 3 — Store partial history

In [14]:
cursor.execute("""
IF COL_LENGTH('Customers', 'PreviousRegion') IS NULL
    ALTER TABLE Customers ADD PreviousRegion NVARCHAR(50)
""")
conn.commit()

# Save current Region as PreviousRegion and update Region
cursor.execute("""
    UPDATE Customers
    SET PreviousRegion = Region,
        Region = ?
    WHERE CustomerID = ?
""", ('North-East', 101))
conn.commit()
print("SCD Type 3: Previous Region stored.")


SCD Type 3: Previous Region stored.


In [15]:
cursor.execute("""
IF OBJECT_ID('Customers_Static') IS NULL
CREATE TABLE Customers_Static (
    CustomerID INT PRIMARY KEY,
    Name NVARCHAR(100),
    Email NVARCHAR(100),
    Membership NVARCHAR(50),
    Region NVARCHAR(50),
    JoinDate DATE
)
""")
cursor.execute("""
IF OBJECT_ID('Customers_History_T4') IS NULL
CREATE TABLE Customers_History_T4 (
    HistoryID INT IDENTITY(1,1) PRIMARY KEY,
    CustomerID INT,
    ChangeDate DATE,
    ChangedField NVARCHAR(50),
    OldValue NVARCHAR(100),
    NewValue NVARCHAR(100)
)
""")
conn.commit()

# Store changes in history table
cursor.execute("""
    INSERT INTO Customers_History_T4 (CustomerID, ChangeDate, ChangedField, OldValue, NewValue)
    VALUES (?, ?, ?, ?, ?)
""", (101, datetime.now().date(), 'Membership', 'Diamond', 'Silver'))
conn.commit()

# Update static table
cursor.execute("""
    UPDATE Customers_Static
    SET Membership = ?
    WHERE CustomerID = ?
""", ('Silver', 101))
conn.commit()
print("SCD Type 4: Separate history logged.")


SCD Type 4: Separate history logged.


In [ ]:
cursor.execute("""
IF OBJECT_ID('Customers_Hybrid', 'U') IS NULL
CREATE TABLE Customers_Hybrid (
    CustomerID INT,
    Name NVARCHAR(100),
    Email NVARCHAR(100),
    Membership NVARCHAR(50),
    Region NVARCHAR(50),
    PreviousRegion NVARCHAR(50),
    JoinDate DATE,
    StartDate DATE,
    EndDate DATE,
    IsCurrent BIT,
    Version INT
)
""")
conn.commit()

# Insert new versioned record, mark old as not current
cursor.execute("UPDATE Customers_Hybrid SET IsCurrent = 0, EndDate = ? WHERE CustomerID = ? AND IsCurrent = 1",
               (datetime.now().date(), 101))
cursor.execute("""
    INSERT INTO Customers_Hybrid (
        CustomerID, Name, Email, Membership, Region, PreviousRegion,
        JoinDate, StartDate, EndDate, IsCurrent, Version
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""", (
    101, 'Deepak Hybrid', 'deepak@hybrid.com', 'Gold', 'West', 'North-East',
    datetime.strptime("2023-01-01", "%Y-%m-%d").date(), datetime.now().date(),
    None, 1, 2
))
conn.commit()
print("SCD Type 6: Hybrid change recorded.")


SCD Type 6: Hybrid change recorded.


<!-- sql code
CREATE TRIGGER trg_BlockJoinDateChange
ON Customers
INSTEAD OF UPDATE
AS
BEGIN
    IF UPDATE(JoinDate)
    BEGIN
        RAISERROR('JoinDate cannot be changed', 16, 1);
        ROLLBACK TRANSACTION;
        RETURN;
    END
    UPDATE Customers
    SET Name = inserted.Name,
        Email = inserted.Email,
        Membership = inserted.Membership,
        Region = inserted.Region
    FROM inserted
    WHERE Customers.CustomerID = inserted.CustomerID;
END -->